### 코드 전체 흐름
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(Input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community # Documnet Loaders
%pip install -qU langchain-text-splitters                    # Text Splitters
%pip install langchain-chroma                                # Vector DB(Chroma)
%pip install -U langchain langchainhub --quiet               # RetrievalQA Chain

In [18]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap=200,  # 텍스트를 분할할 때, 각 chunk(조각) 사이에 중복되는 부분의 토큰 수
)

loader = Docx2txtLoader('./tax.docx')
document = loader.load()                                           #단순히 문서를 읽을 때
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서를 쪼갤 때

In [19]:
print('쪼개기 전 문서 : ',len(document))
print('쪼갠 문서 : ',len(document_list))

쪼개기 전 문서 :  1
쪼갠 문서 :  220


In [20]:
from dotenv import load_dotenv                # OpenAI API Key 사용을 위해 환경변수 등록
from langchain_openai import OpenAIEmbeddings # OpenAI 의 Embedding을 사용

load_dotenv() # 환경변수 불러오기

embeddings = OpenAIEmbeddings(model='text-embedding-3-large') # 기본모델은 002인 예전모델이다. 신규 모델을 사용하기 위해 추가

Python-dotenv could not parse statement starting at line 2


In [21]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=document_list, embedding=embeddings, collection_name='chroma_tax',persist_directory="./chroma") # 쪼개놓은 데이터와 임베딩 모델을 넣어준다.

Python-dotenv could not parse statement starting at line 2


In [22]:
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, 1); # similarity_search() 함수는 유사도 검색을 위한 함수이다.

In [23]:
retrieved_docs

[Document(metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [25]:
# 기존에는 질문만 날렸지만 이제는 질문+문서 를 질의한다. 즉, 프롬프트 작성
prompt = f"""[Identity]
- 당신은 현존하는 세계 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""


In [26]:
ai_message = llm.invoke(prompt)

In [27]:
ai_message.content

'연봉 3000만원인 직장인의 소득세를 계산하기 위해서는 과세표준을 계산하고, 이에 해당하는 세율을 적용해야 합니다. 다만, 소득세는 다양한 공제 항목과 세율 체계가 적용되므로 정확한 소득세 금액을 계산하기 위해서는 추가적인 정보가 필요할 수 있습니다. 기본적인 소득세 계산 과정을 설명드리겠습니다.\n\n1. **과세표준 계산**: 연봉에서 각종 공제를 차감하여 과세표준을 계산합니다. 예를 들어, 국민연금, 건강보험료, 고용보험료 등의 사회보험료와 기본공제, 인적공제 등이 있습니다.\n\n2. **세율 적용**: 한국의 소득세는 누진세율 구조를 가지고 있습니다. 2023년 기준으로 개인 소득세율은 다음과 같습니다:\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 15%\n   - 4,600만원 초과 ~ 8,800만원 이하: 24%\n   - 8,800만원 초과 ~ 1억 5천만원 이하: 35%\n   - 1억 5천만원 초과 ~ 3억원 이하: 38%\n   - 3억원 초과 ~ 5억원 이하: 40%\n   - 5억원 초과: 42%\n\n3. **세액 계산**: 과세표준에 해당하는 세율을 적용하여 산출세액을 계산합니다. 필요에 따라 세액공제 등이 추가로 적용될 수 있습니다.\n\n구체적인 소득세 금액을 계산하기 위해서는 근로소득공제, 인적공제, 특별공제 등을 고려해야 합니다. 이러한 공제 항목은 개인의 상황에 따라 다를 수 있으므로, 소득세 계산기를 사용하거나 세무 전문가의 도움을 받는 것이 좋습니다.'

In [28]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

C:\Users\DataLogics\miniconda3\envs\llm_lab\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [30]:
# QA Chain 만들기
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [31]:
ai_message = qa_chain({"query" : query})

In [33]:
ai_message

{'query': '연봉 3000만원인 직장인의 소득세는 얼마인가요?', 'result': "I don't know."}